# Red card

Government of a city gives a red ID card to its residents. Process of getting a red card involves N stages. At each stage, a govt official will check if application papers submitted by resident is OK. The govt has appointed M officials to handle papers at each stage.

So there are MN officials in all. The govt has grouped the MN officials into M channels (each channel containing N officials, each official is responsible for one stage). Each official takes some number of days to process the paperwork at his/her stage in his/her channel

An applicant may choose any channel and can also switch from channel i to channel i+1 (or channel M to channel 1) as he clears the N stages. But he can only switch in between stages and not during a stage. There is no limit on the number of channel changes.

Find minimum number of days required for an applicant to clear all stages.

Sample input: 4 stages, 3 channels

Channel 1 : 2 6 1 8

Channel 2 : 3 6 2 6

Channel 3 : 4 2 3 6


Answer : If applicant sticks to channel 1, he will take 2 + 6 + 1 + 8 =  17 days.  But he can start with channel 2 (stage 1), shift to channel 3 (stage 2), shift to channel 1 (stage 3) and end with channel 2 (stage 4). He then takes
3 + 2 + 1+ 6 = 12 days. 12 turns out to be the minimum number of days required


# Approach

Create a DAG

**Vertices** : 

{(channel i, pre-stage j)} for 0<=i<=M-1 and 0<= j<= N



**Edges** : 

- (channel i, pre-stage j) ---> (channel i, pre-stage j+1) with weight (channel i, stage j number of days)  for 0<= i <= M-1, 0<= j <= N-1



- (channel i, pre-stage j) ---> (channel i+1, pre-stage j+1) with weight (channel i, stage j number of days) for 0<= i < M-1, 0<= j <= N-1


- (channel M-1, pre-stage j) ---> (channel 0, stage j+1) with weight (channel M-1, stage j number of days)  for 0<= j <= N-1


This is a DAG because you can only go to increasing stages, so can't loop back from stage j to stage less than j.


Need to find the shortest path joining (*, pre-stage 0) and (**, pre-stage N) where * and ** can be anything.






In [0]:
import pprint
from typing import List
from collections import defaultdict, deque
from math import inf 
class Solution:
    def minStages(self, arr: List[List[int]]) -> int:
        #create DAG
        
        M = len(arr)
        N = len(arr[0])

        vertices = [(i, j) for i in range(M) for j in range(N+1)]
        adjacency_dict = defaultdict(list) 
        predec_dict = defaultdict(list)
        indegree_list = [[0]*(N+1) for i in range(M)]
        
        #update adjacency dict, predec dict and initialize indegree list simultaneously
        for i, j in vertices:
          if j < N:
            adjacency_dict[(i,j)].append((i,j+1, arr[i][j]))
            predec_dict[(i,j+1)].append((i,j, arr[i][j]))
            indegree_list[i][j+1] += 1
            if i < M-1:
                adjacency_dict[(i,j)].append((i+1,j+1, arr[i][j]))
                predec_dict[(i+1,j+1)].append((i,j, arr[i][j]))
                indegree_list[i+1][j+1] += 1
            else:
                adjacency_dict[(i,j)].append((0,j+1, arr[i][j]))
                predec_dict[(0,j+1)].append((i,j, arr[i][j]))
                indegree_list[0][j+1] += 1

        #pprint.pprint(adjacency_dict)
        #pprint.pprint(predec_dict)
        #pprint.pprint(indegree_list)

        
        #create queue of indegree 0 vertices
        indegree_0_queue = deque([(i, 0) for i in range(M)])
        no_of_degree_0_initially = len(indegree_0_queue)
        #pprint.pprint(indegree_0_queue)


        topologically_sorted_vertices = []
        #topological sort
        while indegree_0_queue:
            processing_vertex = indegree_0_queue.popleft()
            topologically_sorted_vertices.append(processing_vertex)
            for neighbour in adjacency_dict[processing_vertex]:
                indegree_list[neighbour[0]][neighbour[1]] -= 1
                if indegree_list[neighbour[0]][neighbour[1]] == 0:
                    indegree_0_queue.append((neighbour[0], neighbour[1]))
                    
        #pprint.pprint(topologically_sorted_vertices)

        #Actually the topologically sorted order is obvious: list (channel 0<= i <= M-1, pre-stage j) in increasing order of j ..


      
        min_pathlength_to_= {}
        
       
        for v in topologically_sorted_vertices:
            if not predec_dict[v]:
              min_pathlength_to_[v] = 0
              continue


            min_pathlength_from_all_initial_vertices = inf
            for predec in predec_dict[v]:
                    min_pathlength_from_all_initial_vertices = min(min_pathlength_from_all_initial_vertices, min_pathlength_to_[(predec[0], predec[1])]+ predec[2]) 
            min_pathlength_to_[v] = min_pathlength_from_all_initial_vertices 
            #print(v, min_pathlength_to_[v])
                
        answer = inf
        for i in range(M):
          answer = min(answer, min_pathlength_to_[(i,N)])

        return answer

            
        
     
           
                

In [0]:
sol = Solution()

In [47]:
sol.minStages([[2, 6, 1, 8], [3, 6, 2, 6], [4, 2, 3, 6]])

12